In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import pathlib

import random

In [2]:
dataPath = '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/'
dataRoot = pathlib.Path(dataPath)

In [3]:
imageDirs = []
for item in dataRoot.iterdir():
    imageDirs.append(item)

dataRoot = []
for path in imageDirs:
    dataRoot.append(pathlib.Path(path))

In [13]:
dataRoot

[PosixPath('/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected'),
 PosixPath('/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized')]

In [4]:
labeledImages = []
type = 0
for path in dataRoot:
    for image in path.iterdir():
        labeledImages.append((type,str(image)))
    type += 1
random.shuffle(labeledImages)

[(0,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C87P48ThinF_IMG_20150820_132514_cell_181.png'),
 (1,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C59P20thinF_IMG_20150803_113809_cell_39.png'),
 (0,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C42P5thin_original_IMG_20150623_134919_cell_55.png'),
 (1,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C186P147NThinF_IMG_20151203_150808_cell_142.png'),
 (1,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C99P60ThinF_IMG_20150918_141129_cell_101.png'),
 (0,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C131P92ThinF_IMG_20151004_145302_cell_29.png'),
 (0,
  '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C134P95ThinF_IMG_20151005_121048_cell_93.png'),
 (1,
  '/s/bach/m/under/epike919/Spri

In [5]:
labeledTrain = []
labeledTest = []
i = 0
for image in labeledImages:
    if(i < .8*len(labeledImages)):
        labeledTrain.append(image)
        i += 1
    else:
        labeledTest.append(image)
trainPaths = [x[1] for x in labeledTrain]
trainLabels = [x[0] for x in labeledTrain]
testPaths = [x[1] for x in labeledTest]
testLabels = [x[0] for x in labeledTest]

In [15]:
trainPaths

['/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C87P48ThinF_IMG_20150820_132514_cell_181.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C59P20thinF_IMG_20150803_113809_cell_39.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C42P5thin_original_IMG_20150623_134919_cell_55.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C186P147NThinF_IMG_20151203_150808_cell_142.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C99P60ThinF_IMG_20150918_141129_cell_101.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C131P92ThinF_IMG_20151004_145302_cell_29.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Uninfected/C134P95ThinF_IMG_20151005_121048_cell_93.png',
 '/s/bach/m/under/epike919/Spring19/CS445/TermProjectStuff/cell_images/Parasitized/C18

In [6]:
def loadAndPreprocessImage(path):
  image = tf.read_file(path)
  return preprocessImage(image)

def preprocessImage(image):
  image = tf.image.decode_png(image, channels=3)
  image = tf.image.resize_images(image, [128, 128])
  image = image / 127.5 - 1.0  # normalize to [-1,1] range

  return image

In [7]:
def prepDataForConvNet(imagePaths, labels, batch_size):
    # Convert to Tensor
    imagePaths = tf.convert_to_tensor(imagePaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer([imagePaths, labels],
                                                 shuffle=True)

    # Read images from disk
    image = loadAndPreprocessImage(image)

    # Normalize
    image = image * 1.0/127.5 - 1.0

    # Create batches
    images, labels = tf.train.batch([image, label], batch_size=batch_size,
                          capacity=batch_size * 8,
                          num_threads=4)
    return images, labels

In [8]:
learning_rate = 0.001
num_steps = 500
batchSize = 128
display_step = 1

# Network Parameters
dropout = 0.75 # Dropout, probability to keep units

# Image Parameters_
N_CLASSES = 2 # CHANGE HERE, total number of classes
IMG_HEIGHT = 128 # CHANGE HERE, the image height to be resized to
IMG_WIDTH = 128 # CHANGE HERE, the image width to be resized to
CHANNELS = 3 # The 3 color channels, change to 1 if grayscale

In [9]:
trainImages, trainLabels = prepDataForConvNet(trainPaths, trainLabels, batchSize)
testImages, testLabels = prepDataForConvNet(testPaths, testLabels, batchSize)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To 

In [10]:
#trainPathDS = tf.data.Dataset.from_tensor_slices(trainPaths)
#trainImageDS = trainPathDS.map(loadAndPreprocessImage, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#trainLabelDS = tf.data.Dataset.from_tensor_slices(tf.cast(trainLabels, tf.int64))
#trainDS = tf.data.Dataset.zip((trainImageDS, trainLabelDS))

#testPathDS = tf.data.Dataset.from_tensor_slices(testPaths)
#testImageDS = testPathDS.map(loadAndPreprocessImage, num_parallel_calls=tf.data.experimental.AUTOTUNE)
#testLabelDS = tf.data.Dataset.from_tensor_slices(tf.cast(testLabels, tf.int64))
#testDS = tf.data.Dataset.zip((testImageDS, testLabelDS))

In [11]:
# Create model
def conv_net(images, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(images, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2
        conv1 = tf.layers.max_pooling2d(conv1, 5, 3)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2 
        conv2 = tf.layers.max_pooling2d(conv2, 5, 3)
        
        conv3 = tf.layers.conv2d(conv2, 128, 3)
        conv3 = tf.layers.max_pooling2d(conv3, 3, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv3)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out

In [12]:
logits_train = conv_net(trainImages, N_CLASSES, dropout, reuse=False, is_training=True)
# Create another graph for testing that reuse the same weights
logits_test = conv_net(trainImages, N_CLASSES, dropout, reuse=True, is_training=False)

#logits_test = conv_net(trainImages, N_CLASSES, dropout, reuse=False, is_training=True)

# Define loss and optimizer (with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels=trainLabels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(trainLabels, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Saver object
saver = tf.train.Saver()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Start the data queue
    tf.train.start_queue_runners()

    # Training cycle
    for step in range(1, num_steps+1):

        if step % display_step == 0:
            # Run optimization and calculate batch loss and accuracy
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        else:
            # Only run the optimization op (backprop)
            sess.run(train_op)

    print("Optimization Finished!")
    # Save your model
    saver.save(sess, 'my_tf_model3')

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Step 1, Minibatch Loss= 0.8624, Training Accuracy= 0.438
Step 2, Minibatch Loss= 7.4597, Training Accuracy= 0.430
Step 3, Minibatch Loss= 0.7419, Training Accuracy= 0.477
Step 4, Minibat

Step 101, Minibatch Loss= 0.6932, Training Accuracy= 0.461
Step 102, Minibatch Loss= 0.6950, Training Accuracy= 0.398
Step 103, Minibatch Loss= 0.6922, Training Accuracy= 0.500
Step 104, Minibatch Loss= 0.6923, Training Accuracy= 0.469
Step 105, Minibatch Loss= 0.6926, Training Accuracy= 0.539
Step 106, Minibatch Loss= 0.6946, Training Accuracy= 0.445
Step 107, Minibatch Loss= 0.6910, Training Accuracy= 0.547
Step 108, Minibatch Loss= 0.6953, Training Accuracy= 0.445
Step 109, Minibatch Loss= 0.6938, Training Accuracy= 0.500
Step 110, Minibatch Loss= 0.6912, Training Accuracy= 0.562
Step 111, Minibatch Loss= 0.6923, Training Accuracy= 0.508
Step 112, Minibatch Loss= 0.6949, Training Accuracy= 0.500
Step 113, Minibatch Loss= 0.6950, Training Accuracy= 0.492
Step 114, Minibatch Loss= 0.6918, Training Accuracy= 0.492
Step 115, Minibatch Loss= 0.6918, Training Accuracy= 0.508
Step 116, Minibatch Loss= 0.6993, Training Accuracy= 0.422
Step 117, Minibatch Loss= 0.6947, Training Accuracy= 0.4

Step 240, Minibatch Loss= 0.6960, Training Accuracy= 0.492
Step 241, Minibatch Loss= 0.6968, Training Accuracy= 0.477
Step 242, Minibatch Loss= 0.6998, Training Accuracy= 0.477
Step 243, Minibatch Loss= 0.6963, Training Accuracy= 0.461
Step 244, Minibatch Loss= 0.6930, Training Accuracy= 0.500
Step 245, Minibatch Loss= 0.6950, Training Accuracy= 0.508
Step 246, Minibatch Loss= 0.6944, Training Accuracy= 0.562
Step 247, Minibatch Loss= 0.6919, Training Accuracy= 0.531
Step 248, Minibatch Loss= 0.6925, Training Accuracy= 0.547
Step 249, Minibatch Loss= 0.6924, Training Accuracy= 0.453
Step 250, Minibatch Loss= 0.6955, Training Accuracy= 0.555
Step 251, Minibatch Loss= 0.6932, Training Accuracy= 0.477
Step 252, Minibatch Loss= 0.6932, Training Accuracy= 0.555
Step 253, Minibatch Loss= 0.6934, Training Accuracy= 0.516
Step 254, Minibatch Loss= 0.6912, Training Accuracy= 0.461
Step 255, Minibatch Loss= 0.6947, Training Accuracy= 0.406
Step 256, Minibatch Loss= 0.6940, Training Accuracy= 0.4

Step 379, Minibatch Loss= 0.6913, Training Accuracy= 0.531
Step 380, Minibatch Loss= 0.6950, Training Accuracy= 0.484
Step 381, Minibatch Loss= 0.6919, Training Accuracy= 0.500
Step 382, Minibatch Loss= 0.6896, Training Accuracy= 0.547
Step 383, Minibatch Loss= 0.6906, Training Accuracy= 0.539
Step 384, Minibatch Loss= 0.6989, Training Accuracy= 0.406
Step 385, Minibatch Loss= 0.6944, Training Accuracy= 0.508
Step 386, Minibatch Loss= 0.6920, Training Accuracy= 0.531
Step 387, Minibatch Loss= 0.6888, Training Accuracy= 0.609
Step 388, Minibatch Loss= 0.6925, Training Accuracy= 0.516
Step 389, Minibatch Loss= 0.6913, Training Accuracy= 0.578
Step 390, Minibatch Loss= 0.6950, Training Accuracy= 0.492
Step 391, Minibatch Loss= 0.6951, Training Accuracy= 0.508
Step 392, Minibatch Loss= 0.6861, Training Accuracy= 0.562
Step 393, Minibatch Loss= 0.6876, Training Accuracy= 0.562
Step 394, Minibatch Loss= 0.6909, Training Accuracy= 0.531
Step 395, Minibatch Loss= 0.6911, Training Accuracy= 0.5

ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[{{node batch_1/fifo_queue_enqueue}}]]


Exception in thread QueueRunnerThread-input_producer_1/input_producer-input_producer_1/input_producer/input_producer_EnqueueMany:
Traceback (most recent call last):
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/anaconda3-4.4.0/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/training/queue_runner_impl.py", line 257, in _run
    enqueue_callable()
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1257, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/usr/local/anaconda/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1407, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Enqueue operation was cancelled
	 [[{{node input_producer_

In [12]:
#LEFT OFF AT PIPE THE DATASET TO A MODEL
#NEED TO:
#    SEPERATE DATA INTO TRAINING AND TESTING ###DONE
#    TRAIN MODEL
#    TEST MODEL